In [6]:
# Runtime ~1min 30sec -- 5000 tweets
# Runtime ~2min -- 10,000 tweets

In [1]:
award_names = ['cecil b. demille award',
 'best motion picture - drama',
 'best performance by an actress in a motion picture - drama',
 'best performance by an actor in a motion picture - drama',
 'best motion picture - comedy or musical',
 'best performance by an actress in a motion picture - comedy or musical',
 'best performance by an actor in a motion picture - comedy or musical',
 'best animated feature film',
 'best foreign language film',
 'best performance by an actress in a supporting role in a motion picture',
 'best performance by an actor in a supporting role in a motion picture',
 'best director - motion picture',
 'best screenplay - motion picture',
 'best original score - motion picture',
 'best original song - motion picture',
 'best television series - drama', 'best performance by an actress in a television series - drama',
 'best performance by an actor in a television series - drama',
 'best television series - comedy or musical',
 'best performance by an actress in a television series - comedy or musical',
 'best performance by an actor in a television series - comedy or musical',
 'best mini-series or motion picture made for television',
 'best performance by an actress in a mini-series or motion picture made for television',
 'best performance by an actor in a mini-series or motion picture made for television',
 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television',
 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']
new_award_names = [0]*len(award_names)
for i in range(len(award_names)):
    new_award_names[i] = award_names[i].lower()

In [2]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict

In [4]:
df = pd.read_json('../gg2013.json')
df = df['text']
df = df[df.str.contains(' present|introduc|announc', case = False)]
df = df[~df.str.contains('think|should|maybe', case = False)]
present_df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)
if present_df.size > 10000:
    present_df = present_df.sample(10000)

print(present_df)

48        Definetely Kate's top designers are present in...
465       @JakeCortes @Jessica_Salado comentando los  Ne...
1176      RT @VictorBejerano: @JakeCortes @Jessica_Salad...
1561      RT @TV3Xpose: What's your verdict on @ENews pr...
1976      THISSSSSSSSSS RT @as_seenontv It's appalling t...
                                ...                        
174317    Congratulations @RealHughJackman  well deserve...
174349    RT @popsugar: Aw Ben introducing the cast back...
174440    RT @VanityFair: Amy Poehler and Tina Fey bid a...
174442    it would have been funnier if they had present...
174485    The  ® have come and gone and we’re proud to a...
Name: text, Length: 2642, dtype: object


<ipython-input-4-220c5f5c64a7>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  present_df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)


In [5]:
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [6]:
pattern1 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "present"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
pattern2 = [{"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}, {"LEMMA": "present"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
pattern3 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "present"}, {"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]
pattern4 = [{"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}, {"LEMMA": "present"}, {"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]

In [7]:
matcher = Matcher(nlp.vocab)
matcher.add("Presenter", [pattern1, pattern2, pattern3, pattern4])

In [8]:
presenters = {}
presenters = defaultdict(lambda: 1, presenters)
for i, text in present_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        presenters[str(span).strip(punctuation).strip()] = presenters[str(span).strip(punctuation).strip()] +1

In [9]:
presenters = sorted(presenters.items(), key=lambda item: item[1], reverse = True)
print(presenters)

[('JuliaRoberts presenting Best Motion Picture', 3), ('Jennifer Lawrence present Best Supporting Actor', 2), ('J.Lo present Best Original Score, Motion', 2), ('Arnold #Schwarzenegger present Best Foreign Film', 2), ('Sylvester presenting Best Foreign Language award', 2), ('Schwarzenegger presenting Best Foreign Film', 2), ('Julia Roberts presents Best Motion Picture', 2)]


In [10]:
import Levenshtein
# tweet = string, awards = dict
def DistMeasure(tweet):
    awards = {}
    tweet = tweet.lower()
    tweet = tweet.replace('tv', 'television')
    tweet = tweet.replace('film', 'motion picture')
    tweet = tweet.replace('movie', 'motion picture')
    for award in new_award_names:
        split_award = award.split()
        award_dist = 0
        
        split_tweet = tweet.split()
        for word in split_tweet:
            array = []
            for word2 in split_award:
                array.append(Levenshtein.distance(word, word2))
            min_val = array[np.argmin(array)]
            award_dist += min_val
        awards[award] = award_dist
        
    val = min(awards.values())
    res = [key for key in awards if awards[key] == val]
    if len(res) > 10:
        res = []
    return res

In [11]:
award_list = [dict() for x in range(len(new_award_names))]

for tweet in presenters: 
    tweet = tweet[0]
    split = tweet.lower().split()
    
    index = split.index("best")

    award_phrase = " ".join(split[index:])
    matching_awards = DistMeasure(award_phrase)
    for a in matching_awards:
        award_index = new_award_names.index(a)  
        person = split[:index-1]
        person = " ".join(person)
        if person not in award_list[award_index]:
            award_list[award_index][person] = 1
        else:
            award_list[award_index][person] = award_list[award_index][person]+1

In [12]:
import operator

i = 0
awards = [[] for x in range(len(new_award_names))]
for award in award_list:
    if award != {}:
        max_val = max(award.items(), key=operator.itemgetter(1))[1]
        j = 0
        list_award = list(award)
        for b in award:
            if award[b] == max_val and len(awards[i]) < 2:
                awards[i].append(list_award[j])
            j+=1
    i+=1
print(awards)


[[], [], [], [], [], [], [], [], ['sylvester'], [], ['jennifer lawrence'], [], [], ['j.lo'], [], [], [], [], [], [], [], [], [], [], [], ['jennifer lawrence']]


In [13]:
result = dict(zip(award_names, awards)) 
print(result)

{'cecil b. demille award': [], 'best motion picture - drama': [], 'best performance by an actress in a motion picture - drama': [], 'best performance by an actor in a motion picture - drama': [], 'best motion picture - comedy or musical': [], 'best performance by an actress in a motion picture - comedy or musical': [], 'best performance by an actor in a motion picture - comedy or musical': [], 'best animated feature film': [], 'best foreign language film': ['sylvester'], 'best performance by an actress in a supporting role in a motion picture': [], 'best performance by an actor in a supporting role in a motion picture': ['jennifer lawrence'], 'best director - motion picture': [], 'best screenplay - motion picture': [], 'best original score - motion picture': ['j.lo'], 'best original song - motion picture': [], 'best television series - drama': [], 'best performance by an actress in a television series - drama': [], 'best performance by an actor in a television series - drama': [], 'bes